In [ ]:
# Uncomment these lines if you have not installed any packages yet
# !pip install vislearnlabpy

To open in Google CoLab: https://colab.research.google.com/github/vislearnlab/vllpy/blob/main/examples/drawings/mongo_retrieval.ipynb

In [ ]:
import os
from pathlib import Path

from PIL import Image
import pymongo as pm
from dotenv import load_dotenv
load_dotenv()
from vislearnlabpy.extractions.drawingtask_extractor import DrawingTaskExtractor
from vislearnlabpy.drawings.svg_render_helpers import StrokeSettings

Fill out the fields below. Also make sure to fill in your username and password into an auth.txt file, using a similar format to how the auth_template.txt file is set up, before running the commands below.

In [ ]:
# make sure to add password to a .env file
DATABASE_NAME = "kiddraw"
COLLECTION_NAME = "cdm_birch_v1" # collections to play around with: "birch_run_v1", "cdm_hybrid_v1", "cdm_run_v8", "india_run_v1"
SAVE_DIR = "birch_draw" # often set to /Volumes/vislearnlab/experiments/drawings/data..

This is the Mongo extractor

In [ ]:
# do not commit this auth.txt file!
auth_file = "auth.txt"
with open(auth_file, "r") as f:
    mongo_connection_string = f.readline().strip()
# use a different extractor object for different collections
birch_extractor = DrawingTaskExtractor(mongo_connection_string, DATABASE_NAME, "birch_run_v1", SAVE_DIR)

Can be used to directly prompt Mongo with PyMongo

In [ ]:
all_images = list(birch_extractor.collection.find({'$and': [{'dataType':'finalImage'}]}))
print(f"Total final images in {birch_extractor.collection.name}: {len(all_images)}")

Or can extract full datasets as needed.

In [ ]:
# transform_file 
birch_extractor.extract_images(transform_file=True)

In [ ]:
birch_extractor.extract_audio()

Extracting strokes! By defualt extracted as png but you can extract gifs as well!

Extracting all whale pngs for BD004. Remove the filter to save all strokes across categories.

In [ ]:
birch_extractor.extract_strokes(save_type="png", category=["a whale"], participant_id=["BD004"])

These are the SVG values for each stroke that can be rendered as png files etc. You can play around with the speed and line width, etc. We don't have overwrite built in yet so you might have to delete the file or change the save directory each time for now, apologies.

In [ ]:
stroke_settings = StrokeSettings(line_width=5, 
                                 # lower stroke duration multiplier is faster! 0.5 is equivalent to 1 second in 500 ms
                                 stroke_duration_multiplier=0.5,
                                 edgecolor='black',
                                 fps=15,
                                 # bigger = higher res
                                 imsize=6)

In [ ]:
birch_extractor.extract_strokes(save_type="png", category=["a whale"], participant_id=["BD004"], stroke_settings=stroke_settings)

Extracting high-res images by using the strokes!

In [ ]:
from dataclasses import replace
# larger image size for high res images
kisumu_extraction_settings_original_color = replace(stroke_settings, imsize=8)
birch_extractor.extract_images(transform_file=True, stroke_settings=stroke_settings, highres=True)